## Inference of dataset XXX by XX

In [ ]:
# imports
import sys
import os
import time
import importlib
sys.path.append('../model training')

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from scipy.interpolate import interp1d as interp

from utils import net_utils as nu
from utils import imgio as iio
from utils import predictor as pr
from utils.histnorm import NormHist
import tensorflow as tf

from time import time as timer
from time import sleep
import shutil

plt.style.use('ggplot')
from threading import Thread
import glob

In [ ]:
nb_start_t = timer()

# Datasets to be processed

In [ ]:
datasets_ids = list(range(281, 289)) # range of tiled datasets, expected to be last folders, and 8 tiles
merged_ds_idx = datasets_ids[-1] +1 

In [ ]:
#tiles_info
tiles_info = {} #id: [[ny, nx], list_of_tile_dataset_ids],
ny, nx = 2, 4
n_tiles = ny*nx


for idx_ofs, tiles_start_idx in enumerate(datasets_ids[::n_tiles]):
    tiles_info[merged_ds_idx+idx_ofs] = [[ny, nx], list(range(tiles_start_idx, tiles_start_idx+n_tiles))]
print('check if the ids are correct CAREFULLY! Type Y if correct.')
print(tiles_info)
a = input()
if a.lower()!='y':
    raise ValueError("Process terminated by user's request: tiles info is NOT correct.")

overlap = 1389 * 0.12  # 12% of width
tile_dx = int(1389 - overlap)
tile_dy = int(1040 - overlap)

In [ ]:
datasets_path = '../datasets/'
datasets_path_proc = 'Q:\\' # path from processing node. Should be consistent with cfg files, should be within closed privvate network

proc_bin_path = 'C:\\VivoFollow\\bin\\'


# Run device

In [ ]:
dev_ids = [0, 1, 2, 3, 4, 5, 6, 7] # list of available GPUs
dev_id = dev_ids[0]

In [ ]:
auto_proc = True

# Fill datasets info

In [ ]:
rds_id = 7 # ref. ds. id

nums=[]
datasets_names = []
fluo_present = []

datasets_tmplts = []
datasetsf_tmplts = []
datasets_normed = []
datasets_hist_normed = []

block_boundaries = []

for ds_idx in datasets_ids:
    # read info file
    info_file_name = datasets_path + '%d/info.txt' % ds_idx
    with open(info_file_name, 'rt') as f:
        ds_name = f.readline()
    
    #list files in dir
    ds_path = datasets_path + '%d/%s/' % (ds_idx, ds_name)
    all_tif_files = [n for n in os.listdir(ds_path) if ds_name in n]
    
    all_sfx = sorted([n.replace(ds_name+'_t', '').replace('.tif', '') for n in all_tif_files])
    last_sfx = all_sfx[-1]
    
    #get # channel, #time points
    has_fluo = 'c' in last_sfx
    
    if has_fluo:
        n_t_c_s = last_sfx.split('c')
        n_t, n_c = [int(s) for s in n_t_c_s]
    else:
        n_c = 1
        n_t = int(last_sfx)
    
    #fill tmplts, num, fluo present, 
    tmpl_t = '_t%0' + '%d' % len('%d' % n_t) + 'd'
    tmpl_c = 'c%0' + '%d' % len('%d' % n_c if has_fluo else '1') + 'd'
    
    dataset_tmplt = ds_path + ds_name + tmpl_t + (tmpl_c % 1 if has_fluo else '') +'.tif'
    datasetf_tmplts = [ds_path + ds_name + tmpl_t + (tmpl_c % ch) +'.tif' for ch in range(2, n_c+1)] if has_fluo else []
    dataset_normed = ds_path + 'normed'
    
    dataset_hist_normed =  datasets_path + '%d/' % ds_idx + 'hist_normed/img_%03d.tif' #png

    #print(n_t, n_c, has_fluo, tmpl_t, tmpl_c, dataset_tmplt, datasetf_tmplts, dataset_normed)
    
    # use num per dataset
    nums.append(n_t)
    datasets_names.append(ds_name)
    fluo_present.append(has_fluo)

    datasets_tmplts.append(dataset_tmplt)
    datasetsf_tmplts.append(datasetf_tmplts)
    datasets_normed.append(dataset_normed)
    
    datasets_hist_normed.append(dataset_hist_normed)
    

    block_info_path = os.path.join(datasets_path, '%d' % ds_idx, 'block_info.txt')
    if os.path.exists(block_info_path):
        with open(block_info_path, 'rt') as f:
            txt = f.readline()
    else:
        txt = ''
    if txt:
        block_boundary = [[int(bi) for bi in b.split(' ')] for b in txt.split('|')]
    else:
        block_boundary = [[0, n_t]]
    block_boundaries.append(block_boundary)
    
al_datasets_tmplts = [datasets_path +'%d'%i+ '/imgs_aligned_all/raw/%03d.png' for i,n in zip(datasets_ids,datasets_names)]
al_datasets_normed = [datasets_path +'%d'%i+ '/imgs_aligned_all/normed' for i,n in zip(datasets_ids,datasets_names)]

# Proc

## MT predictor

In [ ]:
class PredictorMT:
    def __init__(self, mod_path, mod_itr,
                 gpu_ids,
                 batch_sz, input_sz,
                 io_map,
                 z_border=2  # one side
                ):
        self.z_border = z_border
        self.gpu_ids = gpu_ids
        self.n_thr = len(self.gpu_ids)
        
        self.predictors = [pr.Predictor(mod_path, mod_itr,
                                        device_id=None,
                                        device_ids=gpu_ids,
                                        gpuid=i,
                                        batch_sz=batch_sz, input_sz=input_sz,
                                        in_out_dict=io_map,
                                       ) for i, dev_id in enumerate(gpu_ids)]
        self.res = []
        
    def normalize_stack(self, stack, norm_stack, normalization_percentile_range):
        return self.predictors[0].normalize_stack(stack, norm_stack, normalization_percentile_range)
    
    def split_stack(self, stack):
        n_z = len(stack)
        n_z_chunk = (n_z + self.n_thr - 1) // self.n_thr
        
        begin_end_pairs = [[max(0, i*n_z_chunk-self.z_border), 
                            min(n_z, (i+1)*n_z_chunk+self.z_border)] for i in range(self.n_thr)]
        
        chunks = [stack[b:e] for b,e in begin_end_pairs]
        return chunks
    
    def merge_result(self, stacks):
        #self.z_border
        
        cropped_overlap = [
                            s[0 if i==0 else self.z_border:
                              len(s) if i==(self.n_thr-1) else -self.z_border
                             ]
                            for i, s in enumerate(stacks)
                          ]
        
        res = np.concatenate(cropped_overlap, axis=0)
        return res
    
    def _predict_image_stack(self, stack_norm, margin, keep_edge, edge_size, thr_idx):
        pred = self.predictors[thr_idx]
        res = pred.predict_image_stack(stack_norm, margin, keep_edge, edge_size)
        self.res[thr_idx] = res
        
    def predict_image_stack(self, stack_norm, margin, keep_edge=True, edge_size=0):
        # split stack
        chunks = self.split_stack(stack_norm)
        
        self.res = [None for i in range(self.n_thr)]
        
        #make threads
        threads = [Thread(target = self._predict_image_stack,
                          args = (chunks[i], margin, keep_edge, edge_size, i)
                         ) for i in range(self.n_thr)]
        
        #run all
        for t in threads:
            t.start()
        
        #wait all
        for t in threads:
            t.join()
        
        merged = self.merge_result(self.res)
        #merge ouput
        #processed.shape == (192, 1077, 1405, 3), i.e. plain np tensor
        
        return merged

## Save constatnt models for prediction

In [ ]:
# load models
# fast model
#mod_path = '../BBB/model_BBB_BN_TV_FCN4_CDC_2D_2019.10.21_14-49/'  # 2D, 5 tf
#mod_itr = 80001

#mod_path = '../BBB/model_BBB_BN_TV_FCN4_CDC_2D_2019.11.11_01-11/'  # 2D, 1 tf (last prod)
#mod_itr = 80001

#mod_path = '../BBB/model_BBB_BN_TV_FCN4_CDC_2D_2019.12.03_19-09/'  # 2D, 1 tf, with new BN (never used)
#mod_itr = 80001

mod_path = '../BBB/model_BBB_BN_TV_FCN4_HN_CDC_2D_2021.04.17_22-54/'  # 2D, 1 tf 2021 best 
mod_itr = 30001

io_map = {'in':'stack:0', 'out':'ModelOutput:0', 'out_channels':[0,1,2]}
batch_sz=8

print('AFTER EXECUTING THIS FIRST TIME FOR A MODEL (Needs single-threaded version!) - RESTART THE KERNEL')
print(mod_path, mod_itr)
# pred_fast = pr.Predictor(mod_path, mod_itr,
#                          device_id=dev_id,
#                          batch_sz=batch_sz, input_sz=[512,512],
#                          in_out_dict=io_map)

pred_fast_mt = PredictorMT(mod_path, mod_itr,
                   gpu_ids=dev_ids,
                   batch_sz=batch_sz, input_sz=[512,512],
                   io_map=io_map)

#mod_path = '../BBB/model_BBB_BN_TV_FCNx16_CDC_2019.10.22_18-55/'
#mod_path = '../BBB/model_BBB_BN_TV_FCNx16_CDC_2019.10.17_17-50/'
#mod_itr = 120001

# mod_path = '../BBB/model_BBB_BN_TV_FCNx16_CDC_2020.01.25_19-22/'  # last prod
# mod_itr = 100001

mod_path = '../BBB/model_BBB_BN_TV_FCNx16_HN_CDC_2021.07.06_16-25/'  # 3D, 2021 best
mod_itr = 35001

io_map = {'in':'stack:0', 'out':'ModelOutput:0', 'out_channels':[0,1,2]}

batch_sz=1

print(mod_path, mod_itr)
# pred = pr.Predictor(mod_path, mod_itr,
#                     device_id=dev_id,
#                     batch_sz=batch_sz, input_sz=[512,512],
#                     in_out_dict=io_map)
pred_mt = PredictorMT(mod_path, mod_itr,
                   gpu_ids=dev_ids,
                   batch_sz=batch_sz, input_sz=[512,512],
                   io_map=io_map)

## Load datasets, save match size, normalize and save normalized

In [ ]:
%%time
for tmpl, nnrm, num in zip(datasets_tmplts, datasets_normed, nums):
    print(tmpl)
    stack = iio.read_image_stack(tmpl, num, 1)
    
    print('saving...')
    if len(stack.shape)==4:
        stack = stack[...,0]
    
    for idx, im in enumerate(stack):
        iio.save_image(im, tmpl % (idx+1))
    
    stack_center = stack[:, 130:-130, 130:-130]
    
    print('normalizing...')
    stack_norm = pr.Predictor.normalize_stack(stack=stack, 
                                              norm_stack=stack_center, 
                                              normalization_percentile_range=(2.5, 97.5))
    print('saving normalized...')
    np.savez(nnrm, stack_norm)

## Generate cell probability maps for alignment and save

In [ ]:
def proc_path(path):
    return path.replace(datasets_path, datasets_path_proc).replace('/','\\').replace('%','%%')

def make_genmask_bat(ds_idx, stck_tmpl, num):
    # gen background mask generation bat file. Alignment pipeline is used
    prog = proc_bin_path+'TimeFramesAligner_64.exe '
    cfg = f'-cfg:{proc_bin_path}TFAligner_bg_mask.cfg '
    

    mask_tmpl = datasets_path +'%d/'%ds_idx+'pred_for_algn/'+'img_%03d.png'
    tgt_dir = datasets_path +'%d/'%ds_idx+'imgs_aligned_all/'
    s_t = proc_path(stck_tmpl)
    m_t = proc_path(mask_tmpl)
    tgd = proc_path(tgt_dir)
    
    cmd = '@echo off\n'
    cmd = 'pushd %~dp0\n'
    cmd += prog+cfg
    cmd += '-savedir:"%s" ' % tgd
    cmd += '-n_itr:0 '
    cmd += '-stack:"%s" ' % s_t
    cmd += '-mask:"%s" ' % m_t
    cmd += '-n_frames:%d ' %  num
    
    cmd += '\n'
    
    cmd += 'popd\n'
        
    bat_file = datasets_path + '%d/'%ds_idx + 'genmask.bat'
    #print(cmd)
    with open(bat_file, 'wt') as f:
        f.write(cmd)

In [ ]:
def gen_collective_genmask(ds_idx_list):
    """
    Generated all genmask bat file: executes individual genmask
    returns: full path to created batfile
    """
    #cmd = '@echo off\n'
    cmd = ''

    for ds_idx in ds_idx_list:
        cmd += 'call %d'%ds_idx+'\\genmask.bat \n'
    bat_file = datasets_path + 'genmask_' + str(ds_idx_list) + '.bat'
    with open(bat_file, 'wt') as f:
        f.write(cmd)
        
    return bat_file

In [ ]:
def start_remote_job(datasets_path, bat_file):
    itr = 0
    while(os.path.exists(datasets_path + 'remote.bat')):
        if itr==0:
            print('Waiting previous remote job to be done...')
        itr += 1
        sleep(5)
    shutil.copy(bat_file, datasets_path + 'remote.bat')

In [ ]:
def show_mdl_smpl(raw, cdc):
    mdl=len(cdc)//2
    _=iio.draw_samples(
        (
            raw[mdl,...,0],
            cdc[mdl,...,0],
            cdc[mdl,...,1],
            cdc[mdl,...,2]
        ), color_range=(0,256)
    )    

In [ ]:
t0 = timer()
for i, (nnrm, num, ds_idx, stck_tmpl) in enumerate(zip(datasets_normed, nums, datasets_ids, datasets_tmplts)):
    # read normalized
    stack_normf = np.load(nnrm+'.npz')
    for stack_norm in stack_normf.values():
        break

    # process
    processed = pred_fast_mt.predict_image_stack(stack_norm, margin=4, edge_size=30)
    
    # visualize
    show_mdl_smpl(stack_norm, processed)
    
    print('saving dataset ', ds_idx, end='\r')
    path = datasets_path + '%d/'%ds_idx + '/pred_for_algn/'
    os.makedirs(path, exist_ok=True)
    for i, img in enumerate(processed):
        pimg = Image.fromarray(img[..., 0])
        pimg.save(path+ 'img_%03d.png'%i, quality=99)
        
    
    # make maskgen bat file and run if needed
    make_genmask_bat(ds_idx, stck_tmpl, num)
    
    if auto_proc:
        bat_file = gen_collective_genmask([ds_idx])
        start_remote_job(datasets_path, bat_file)

if not auto_proc:
    for ds_idx, stck_tmpl, num in zip(datasets_ids, datasets_tmplts, nums):
        make_genmask_bat(ds_idx, stck_tmpl, num)
    bat_file = gen_collective_genmask(datasets_ids)
    
t1 = timer()
print(t1 - t0, 's')

## Generate Histogram normalized files

In [ ]:
# wait masks done
i = len(datasets_tmplts) - 1
idx = datasets_ids[i]
test_file = datasets_path +'%d/'%idx+'imgs_aligned_all/bin_mask_bg/%03d.png' % (nums[i]-1)

while not os.path.exists(test_file):
    time.sleep(5)
    
time.sleep(10)

In [ ]:
nh = NormHist('../data/data_prep/ref_imgs/%02d' % rds_id, dev=0)

In [ ]:
%%time
for i, tmpl, tmpl_hn, boundaries, num in zip(datasets_ids, datasets_tmplts, datasets_hist_normed, block_boundaries, nums):
    print(tmpl)
    stack = iio.read_image_stack(tmpl, num, 1)
    if len(stack.shape)==4:
        stack = stack[...,0]
        
    stack_normed_blocks = []
    
    mask_tmpl = datasets_path +'%d/'%i+'imgs_aligned_all/bin_mask_bg/%03d.png'

    
    for idx, block_boundary in enumerate(boundaries):
        bl_begin, bl_end = block_boundary
        stack_src = stack[bl_begin: bl_end]
        stack_src_ref = stack_src[-5:] if idx==0 else stack_src[:5]
        print('normalizing block', idx, 'range', block_boundary, '...', end='\r')
        
        mask_start_idx = (bl_end - 5) if idx==0 else bl_begin
        mask_ref = iio.read_image_stack(mask_tmpl, 5, mask_start_idx)
        if len(mask_ref.shape)==4:
            mask_ref = mask_ref[...,0]
            
        stack_normed_block = nh.correct_stack(stack_src, stack_src_ref, mask_ref)
        #fig, ax = plt.subplots(1, 2, figsize=(12, 4))
        #ax[0].plot(nh.last_lut)
        #ax[1].plot(nh.last_lut[200:])
        #plt.show()
        stack_normed_blocks.append(stack_normed_block)

        
    stack_normed_blocks = np.concatenate(stack_normed_blocks, axis=0)
        
        

    print('saving...                                     ', end='\r')
    os.makedirs(os.path.dirname(tmpl_hn), exist_ok=True)
    for idx, im in enumerate(stack_normed_blocks):
        iio.save_image(im, tmpl_hn % idx)

## Generate bat file for alignment

In [ ]:
# gen alignment  bat file
prog = proc_bin_path+'TimeFramesAligner_64.exe '
cfg = f'-cfg:{proc_bin_path}TFAligner.cfg '


rmdir_statement = lambda path: """IF exist %s (
  rmdir /s /q %s
)\n""" % (path, path)

for ds_idx, stck_tmpl, stck_aux, has_aux, num  in zip(datasets_ids, datasets_hist_normed, datasetsf_tmplts, fluo_present, nums):
    mask_tmpl = datasets_path +'%d/'%ds_idx+'pred_for_algn/'+'img_%03d.png'
    tgt_dir = datasets_path +'%d/'%ds_idx+'imgs_aligned_all/'
    s_t = proc_path(stck_tmpl)
    m_t = proc_path(mask_tmpl)
    tgd = proc_path(tgt_dir)
    
    cmd = '@echo off\n'
    cmd = 'pushd %~dp0\n'
    cmd += prog+cfg
    cmd += '-savedir:"%s" ' % tgd
    cmd += '-n_itr:3 '
    cmd += '-stack:"%s" ' % s_t
    cmd += '-raw_idx_0:0 '
    cmd += '-mask:"%s" ' % m_t
    cmd += '-n_frames:%d ' %  num
    
    if has_aux:
        for aux_id, aux_tmpl in enumerate(stck_aux):
            s_t = proc_path(aux_tmpl)
            cmd += '-stack_%d:"%s" ' % (aux_id, s_t)
            cmd += '-stack_%d_subpixel ' % aux_id
            cmd += '-stack_%d_start:1 ' % aux_id
        
    cmd += '\n'
    
    cmd += rmdir_statement('"%sraw"'%tgd)
    cmd += 'move "%scorrected" "%sraw"\n' % (tgd, tgd)

    if has_aux:
        for aux_id, _ in enumerate(stck_aux):
            cmd += rmdir_statement('"%sflr%d"'%(tgd, aux_id+1))
            cmd += 'move "%scorrected_st_%d" "%sflr%d"\n' % (tgd, aux_id, tgd, aux_id+1)
    cmd += 'popd\n'
        
    bat_file = datasets_path + '%d/'%ds_idx + 'align.bat'
    #print(cmd)
    with open(bat_file, 'wt') as f:
        f.write(cmd)

In [ ]:
# gen all alignments bat file
cmd = '@echo off\n'

for ds_idx in datasets_ids:
    cmd += 'call %d'%ds_idx+'\\align.bat \n'
    
first_last = str(datasets_ids[0])
if len(datasets_ids)>1:
    first_last += '-'+str(datasets_ids[-1])
bat_file = datasets_path + 'align_[' + first_last + '].bat'
with open(bat_file, 'wt') as f:
    f.write(cmd)

In [ ]:
if auto_proc:
    while(os.path.exists(datasets_path + 'remote.bat')):
        sleep(5)
    shutil.copy(bat_file, datasets_path + 'remote.bat')

## Load aligned datasets, normalize and save normalized

In [ ]:
%%time
# wait alignment done
i = len(al_datasets_tmplts) - 1
test_file = al_datasets_tmplts[i]%(nums[i]-1)  # last dataset, first file
while not os.path.exists(test_file):
    time.sleep(5)
    
time.sleep(10)

In [ ]:
%%time
# normalization
for tmpl, nnrm, num in zip(al_datasets_tmplts, al_datasets_normed, nums):
    print(tmpl)
    stack = iio.read_image_stack(tmpl, num)
    stack_center = stack[:, 130:-130, 130:-130]
    stack_norm = pr.Predictor.normalize_stack(stack=stack, 
                                              norm_stack=stack_center, 
                                              normalization_percentile_range=(2.5, 97.5)
                                             )
    np.savez(nnrm, stack_norm)

# Prediction by aligned

In [ ]:
t0 = timer()
for i, (nnrm, ds_idx) in enumerate(zip(al_datasets_normed, datasets_ids)):
    # load normalized
    stack_normf = np.load(nnrm+'.npz')
    for stack_norm in stack_normf.values():
        break
    # process
    processed = pred_mt.predict_image_stack(stack_norm, margin=8, edge_size=60)
    
    #save processed
    print(ds_idx, end='\r')
    pref = os.path.join(datasets_path,'%d'%ds_idx)
    path_cell = os.path.join(pref,'pred_cdc', 'cell')
    path_diap = os.path.join(pref,'pred_cdc', 'diap')
    path_cntr = os.path.join(pref,'pred_cdc', 'cntr')
    path_cntC = os.path.join(pref,'pred_cdc', 'cntC')
    
    os.makedirs(path_cell, exist_ok=True)
    os.makedirs(path_diap, exist_ok=True)
    os.makedirs(path_cntr, exist_ok=True)
    os.makedirs(path_cntC, exist_ok=True)
    
    for i, img  in enumerate(processed):
        for t, path in enumerate([path_cell, path_diap, path_cntr]):
            pimg = Image.fromarray(img[..., t])
            im_path = os.path.join(path, 'img_%03d.png'%i)
            pimg.save(im_path , quality=99)
            
        cell_im = img[..., 0]
        cntr_im = img[..., 2].copy()
        mask_no_cell = cell_im <= 85
        cntr_im[mask_no_cell] = 0

        pimg = Image.fromarray(cntr_im)
        im_path = os.path.join(path_cntC, 'img_%03d.png'%i)
        pimg.save(im_path , quality=99)
        
    del processed
    del stack_norm
        
        
t1 = timer()
print(t1 - t0, 's')

# Tile merging

In [ ]:
merged_tiles_ids = list(tiles_info.keys())

In [ ]:
#gen lists
for merged_idx, ti in tiles_info.items():
    ny, nx = ti[0]
    tile_ids = ti[1]
    
    info_path = os.path.join(datasets_path, '%d' % merged_idx, 'tiles_info')
    os.makedirs(info_path, exist_ok=True)
    
    lists = ['', '', '', '', '', '']
    
    for idx in tile_ids:
        lists[0] += datasets_path_proc + '%d' % idx + r'\imgs_aligned_all\raw\%03d.png'+'\n'
        lists[1] += datasets_path_proc + '%d' % idx + r'\pred_cdc\cell\img_%03d.png'+'\n'
        lists[2] += datasets_path_proc + '%d' % idx + r'\pred_cdc\diap\img_%03d.png'+'\n'
        lists[3] += datasets_path_proc + '%d' % idx + r'\pred_cdc\cntC\img_%03d.png'+'\n'
        lists[4] += datasets_path_proc + '%d' % idx + r'\imgs_aligned_all\flr1\%03d.png'+'\n'
        lists[5] += datasets_path_proc + '%d' % idx + r'\imgs_aligned_all\flr2\%03d.png'+'\n'
        
    tmap = '%d %d\n' % (nx, ny)
    idx = -1
    for iy in range(ny):
        for ix in range(nx):
            idx += 1
            
            tmap += '%d %d %d %d %d\n' % (idx, ix, iy, ix * tile_dx, iy*tile_dy)
            
    for tlist, fname in zip(lists, ['raw.tl', 'cell.tl', 'diap.tl', 'cntc.tl', 'flr1.tl', 'flr2.tl']):
        fpath = os.path.join(info_path, fname)
        with open(fpath, 'wt') as f:
            f.write(tlist)
            
    fpath = os.path.join(info_path, 'map.tm')
    with open(fpath, 'wt') as f:
        f.write(tmap)

In [ ]:
# gen merging bat file
prog_tal = proc_bin_path + 'proc_iv.bat'


ds_id_to_idx = {i:idx for idx, i in enumerate(datasets_ids)}


for idx in tiles_info:
    cmd = '@echo off\n'
    cmd = 'pushd '+proc_bin_path+'\n'
    
    tile_idx = ds_id_to_idx[tiles_info[idx][1][0]]
    n_tf = nums[tile_idx]
    cmd += 'call ' + prog_tal + ' %d' % idx  + ' %d' % n_tf + ' %d' % len(datasetsf_tmplts[tile_idx])
    cmd += '\n'
    cmd += 'popd\n'

    bat_file = datasets_path + '%d/'%idx + 'merge.bat'
    with open(bat_file, 'wt') as f:
        f.write(cmd)

In [ ]:
# gen all merging bat file
cmd = '@echo off\n'

for ds_idx in tiles_info:
    cmd += 'call %d'%ds_idx+'\\merge.bat \n'
bat_file = datasets_path + 'merge_' + str(merged_tiles_ids) + '.bat'
with open(bat_file, 'wt') as f:
    f.write(cmd)

In [ ]:
if auto_proc:
    while(os.path.exists(datasets_path + 'remote.bat')):
        sleep(5)
    shutil.copy(bat_file, datasets_path + 'remote.bat')

# Cell segmentation

In [ ]:
# gen segmentation bat file
prog_seg = proc_bin_path + 'proc_ds_flr_n.bat'

for i, idx in enumerate(datasets_ids+merged_tiles_ids):
    test_idx = i if idx not in merged_tiles_ids else ds_id_to_idx[tiles_info[idx][1][0]]
    has_flour = fluo_present[test_idx]
    
    cmd = '@echo off\n'
    cmd = 'pushd '+proc_bin_path+'\n'
    cmd += 'call ' + prog_seg + ' %d' % idx  + ' %d' % nums[test_idx]+ ' %d' % len(datasetsf_tmplts[test_idx])
    cmd += '\n'
    cmd += 'popd\n'

    bat_file = datasets_path + '%d/'%idx + 'segment.bat'
    with open(bat_file, 'wt') as f:
        f.write(cmd)

In [ ]:
# gen all segmentation bat file
cmd = '@echo off\n'

for ds_idx in datasets_ids:
    cmd += 'call %d'%ds_idx+'\\segment.bat \n'

first_last = str(datasets_ids[0])
if len(datasets_ids)>1:
    first_last += '-'+str(datasets_ids[-1])

bat_file_t = datasets_path + 'segment_[' + first_last + '].bat'
with open(bat_file_t, 'wt') as f:
    f.write(cmd)
    
cmd = '@echo off\n'

for ds_idx in merged_tiles_ids:
    cmd += 'call %d'%ds_idx+'\\segment.bat \n'

first_last = str(merged_tiles_ids[0])
if len(merged_tiles_ids)>1:
    first_last += '-'+str(merged_tiles_ids[-1])
bat_file_m = datasets_path + 'segment_[' + first_last + '].bat'
with open(bat_file_m, 'wt') as f:
    f.write(cmd) 

In [ ]:
# process tiles
if auto_proc:
    while(os.path.exists(datasets_path + 'remote.bat')):
        time.sleep(5)
    shutil.copy(bat_file_t, datasets_path + 'remote.bat')

In [ ]:
%%time
# wait merging and segmnet merged
if auto_proc:
    while(os.path.exists(datasets_path + 'remote.bat')):
        time.sleep(5)
    shutil.copy(bat_file_m, datasets_path + 'remote.bat')

# Cell export

In [ ]:
# wait segmentation done
test_file = datasets_path + '%d/segmentation/cells/cells.gtp' % datasets_ids[-1]  # last dataset, cells file
while not os.path.exists(test_file):
    time.sleep(5)
    
time.sleep(10)

In [ ]:
%%time
# wait segmentation done
test_file = datasets_path + '%d/segmentation/cells/cells.gtp' % merged_tiles_ids[-1]  # last dataset, cells file
while not os.path.exists(test_file):
    time.sleep(5)
    
time.sleep(10)

In [ ]:
# gen export bat file
path_tra = f'call {proc_bin_path}proc_export.bat'

for i, idx in enumerate(datasets_ids + merged_tiles_ids):
    cmd = 'pushd %~dp0\n'
    cmd += path_tra + ' %d' % idx
    cmd += '\n'
    cmd += 'popd\n'

    bat_file = datasets_path + '%d/'%idx + 'export.bat'
    with open(bat_file, 'wt') as f:
        f.write(cmd)

In [ ]:
sel_merged_tiles_ids = merged_tiles_ids # can have subrange

In [ ]:
# gen all export bat file
cmd = '@echo off\n'

for ds_idx in datasets_ids:
    cmd += 'call %d'%ds_idx+'\\export.bat \n'
    
first_last = str(datasets_ids[0])
if len(datasets_ids)>1:
    first_last += '-'+str(datasets_ids[-1])

bat_file_t = datasets_path + 'track_[' + first_last + '].bat'
with open(bat_file_t, 'wt') as f:
    f.write(cmd)
    
cmd = '@echo off\n'

for ds_idx in sel_merged_tiles_ids:
    cmd += 'call %d'%ds_idx+'\\export.bat \n'
    
    
first_last = str(sel_merged_tiles_ids[0])
if len(sel_merged_tiles_ids)>1:
    first_last += '-'+str(sel_merged_tiles_ids[-1])
bat_file_m = datasets_path + 'track_[' + first_last + '].bat'
with open(bat_file_m, 'wt') as f:
    f.write(cmd)
    

In [ ]:
if auto_proc:
    while(os.path.exists(datasets_path + 'remote.bat')):
        time.sleep(5)
    shutil.copy(bat_file_t, datasets_path + 'remote.bat')

In [ ]:
%%time
if auto_proc:
    while(os.path.exists(datasets_path + 'remote.bat')):
        time.sleep(5)
    shutil.copy(bat_file_m, datasets_path + 'remote.bat')

In [ ]:
%%time
if auto_proc:
    while(os.path.exists(datasets_path + 'remote.bat')):
        time.sleep(5)

In [ ]:
nb_end_t = timer()
print(f'notebook run time: {(nb_end_t - nb_start_t):.2f} sec')